**Hey Luit, hier alvast een opzet van de opdracht. Ik zal per blokje kort even aangeven of het boeiend voor jou is.**

Dit is **niet** belangrijk voor jouw onderdelen, dit is alleen hier om het inladen van bestanden te fixen

In [78]:
def format_add(format_list, input_data, target_data):
    ### Constructs a dictionary using input_data and 
    ### the format_list as the dictionary format and 
    ### appends the dictionary to the target data.
    
    appender_dict = {}
    
    for count, item in enumerate(format_list):
        try:
            appender_dict.update({item: input_data[count]})
        except IndexError:
            appender_dict.update({item: None})
            
    target_data.append(appender_dict)

def build_file(format_list, file_path, delimiter):
    ### Reads a file and converts the delimiter separated data 
    ### in the file, based on the format provided by format_list, 
    ### into a dictionary list.
    
    new_list = []
    
    with open(file_path) as file:
        print(f"Unpacking: '{file_path}'")
    
        for line in file:
            stripped_line = line.strip().split(delimiter)
            format_add(format_list, stripped_line, new_list)
        
    print("Unpacked succesfully")
    
    return new_list

Hier staan de drie belangrijke datasets.

```user_data``` is een lijst van dictionaries waar elke dict een regel is uit de oorspronkelijke data bestanden. De keys zie je in de lijst van ```build_file()```. ```movie_data```en ```rating_data``` hebben het zelfde princiepe.

Als je ```user_data[0]``` gebruikt, krijg je dus een hele regel. Let op, UIDs en ```[id]``` zijn mogelijk niet 1 op 1! (zie ```find_on_key_and_value``` voor makkelijker zoeken naar gebruikers/movies/ratings/andere dingen).

In [93]:
file_path_users = "./ml-1m/users.dat"
file_path_movies = "./ml-1m/movies.dat"
file_path_ratings = "./ml-1m/ratings.dat"

user_data = build_file(
    ["uid", "gender", "age", "occupation", "zip-code"],
    file_path_users,
    "::"
)

movie_data = build_file(
    ["mid", "title", "genre"],
    file_path_movies,
    "::"
)

rating_data = build_file(
    ["uid", "mid", "rating", "timestamp"],
    file_path_ratings,
    "::"
)

Unpacking: './ml-1m/users.dat'
Unpacked succesfully
Unpacking: './ml-1m/movies.dat'
Unpacked succesfully
Unpacking: './ml-1m/ratings.dat'
Unpacked succesfully


Gebruik dit om een entry te zoeken gebaseerd op een key (zoals *uid*, *gender*, *title*) en de verwachtte waarde. **Let op** dit retourneerd een lijst (kunnen dus meerdere entries zijn), maar als je maar 1 ding zoekt doe je gewoon: ```find_on_key_and_value(blah, blah, blah)[0]``` of ```find_on_key_and_value(blah, blah, blah)[:1]``` (als er niets gevonden is krijg je een lege lijst)

In [97]:
def find_on_key_and_value(target_data, key, value):
    ### Filter the data based on a key-value combination in the list of dictionaries.
    return list(filter(lambda obj : obj[key] == value, target_data))

Hier een algemeen gemiddelde, niet zo spannend denk ik, maar misschien nodig bij de twee grotere methodes.

In [80]:
from numpy import mean

# Calculate the mean over a comprehenced list based 
# on the rating fields in rating_data to get a global mean. 
all_rating_mean = mean([int(item["rating"]) for item in rating_data])

Hier moet je vooral niet kijken naar wat de code is, want comprehension is lekker niet zo leesbaar, maar dit zoekt alle gemiddelden voor een bepaalde ```target_mean``` gebaseerd op een ```target_key```.

Dus: ```bind_mean(rating_data, user_data, "uid", "rating", all_rating_mean)``` zoekt voor alle gebruikers (*uid*) de gemiddelde rating die die gebruiker heeft gegeven en maakt vervolgens een dictionary met daarin de gemiddelde van die gebruikers gekoppeld aan de uid van die gebruiker.

Oh ja, **let op**, de dataset is nogal groot, dus let er op dat dit even kan duren voordat het klaar is (5 minuten ofzo).

In [81]:
from numpy import mean, isnan

def bind_mean(bind_data_source, bind_data_target, target_key, target_mean, default=None):
    ### Creates a dictionary of target_keys with their respective means. 
    ### If the target key does not exist in the source_data, then a default value will be set.
    return {l_obj[target_key]: x if not isnan(x := mean([int(item[target_mean]) for item in filter(lambda _r : _r[target_key] == l_obj[target_key], bind_data_source)])) else default for l_obj in bind_data_target}

Deze stap duurt dus waarschijnlijk rond de 5 minuten, maar je hoeft dit maar één keer te doen gelukkig.

In [82]:
# Generate a dictionary of user_ids with the corresponding means of ratings that a user has given. 
# If the user does not exist in the ratings file, the global average rating is filled in instead.
user_rating_mean = bind_mean(rating_data, user_data, "uid", "rating", all_rating_mean)

# Generate a dictionary of movie_ids with the corresponding means of ratings that a movie has received. 
# If the movie does not exist in the ratings file, the global average rating is filled in instead.
movie_rating_mean = bind_mean(rating_data, movie_data, "mid", "rating", all_rating_mean)

Hier wat voorbeelden van wat je van de data en functies kunt verwachten

In [135]:
print(user_data[0]) # First line in user_data file
print(find_on_key_and_value(user_data, "uid", "80")) # Line based on uid=80
print(find_on_key_and_value(rating_data, "rating", "5")[:3]) # First three lines based on rating=5
print(user_rating_mean["2"]) # The average score of movies given by the user with uid=2
print(movie_rating_mean["6"]) # The average score of a movie mid=5
print(all_rating_mean) # Average of all scores

{'uid': '1', 'gender': 'F', 'age': '1', 'occupation': '10', 'zip-code': '48067'}
[{'uid': '80', 'gender': 'M', 'age': '56', 'occupation': '1', 'zip-code': '49327'}]
[{'uid': '1', 'mid': '1193', 'rating': '5', 'timestamp': '978300760'}, {'uid': '1', 'mid': '2355', 'rating': '5', 'timestamp': '978824291'}, {'uid': '1', 'mid': '1287', 'rating': '5', 'timestamp': '978302039'}]
3.7131782945736433
3.0
3.581564453029317
